# DewanLab DeepLabCut Video Analysis 

In [ ]:
import os
from pathlib import Path
os.environ['DLClight']="True"
import deeplabcut

In [ ]:
## User Configuratbles

video_dir = '/blue/adamdewan/EPM/'
video_file_extensions = ['avi', 'mp4', 'mkv']
recursive_video_search = True

_shuffle = 1
_save_as_csv = True
_trailpoints = 0

In [ ]:
# The notebook should be in the root directory of the project
current_dir = Path.cwd()
config_path = current_dir.joinpath('config.yaml')
new_videos_dir = Path(video_dir)

if not new_videos_dir.exists():
    raise FileNotFoundError(f'The path {{{new_videos_dir}}} does not exist!')

# Get a list of the videos in "New Video Files"
video_paths = []

for file_extension in video_file_extensions:
    search_string = f'*.{file_extension}'
    if recursive_video_search:
        new_vids = list(new_videos_dir.rglob(search_string))
    else:
        new_vids = list(new_videos_dir.glob(search_string))

    if len(new_vids) > 0:
        video_paths.extend(new_vids)
        
video_paths = [video for video in video_paths if 'labeled' not in video.name]
# Filter out any analyzed videos

In [ ]:
for video in video_paths:
    video_type = video.suffix
    analyze_videos(str(config_path), str(video), shuffle=_shuffle, save_as_csv=_save_as_csv, videotype=video_type)
    create_labeled_video(str(config_path), str(video), shuffle=_shuffle, videotype=video_type, trailpoints=)_trailpoints